<hr style=' border: 5px solid red'>
<hr style=' border: 5px solid black'>
<hr style=' border: 5px solid red'>

<h1 style="text-align:center;"> Finnish scraping project</h1>
<h3 style="text-align:center;"> Phase 1: Scraping Finnish verbs</h3>

<hr style=' border: 5px solid red'>
<hr style=' border: 5px solid black'>
<hr style=' border: 5px solid red'>
    

<p> The aim of this notebook is to generate a data set to be used to build a database that will be used as a backend for a language learning app for my own personal use.

The notebook will use bs4 url lib and er libraries to scrape public sites for Finnish vocabulary. it will then be processed using powerQuery and other tools before being transferred to a postgres database.
</p>

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import urllib as url
from IPython.core.display import HTML
import time #set sleep to avoid url blocking scraper
import pygwalker as pyg
import keyboard

ModuleNotFoundError: No module named 'pygwalker'

<hr style=' border: 2px solid red'>
<hr style=' border: 2px solid black'>
<h4 style="text-align:center;"> Stage 1</h4>
<hr style=' border: 2px solid black'>
<hr style=' border: 2px solid red'>

<p> first step is to pull the list of all infintives of Finnish verbs from <a href = "https://cooljugator.com/fi/list/all">this site </a></p>

In [ ]:
# load the site using URL lib
doc = url.request.urlopen('https://cooljugator.com/fi/list/all').read()

# read the page using bs4
soup = bs(doc,'html.parser')

#use prettify to make content more readable
soup.prettify()

# all 
links = soup.find_all('li')
# for i in range(41,50):
#     print(links[i])
verbs = [str(x).split(sep = '>')[1] for x in links]
verbs = [str(x).split(sep = '/')[-1] for x in verbs]
verbs = [str(x).replace('"','') for x in verbs]

translations = [str(x).split(sep = '>')[3] for x in links]
translations = [str(x).replace('- ','') for x in translations]
translations = [str(x).replace('</li','').lstrip() for x in translations]



translations
# for i in range(0,10):
#     print(verbs[i])
    
    
verb_df = pd.DataFrame({'Finnish infinitive':verbs,'English inifinitive':translations})
#verb_df.columns = ['infinitive','English']
verb_df = verb_df[:8092]
#verb_df.tail()

verb_df.to_csv("./cleaneddb/verbinfinitives.csv", index = False)

<hr style=' border: 2px solid red'>
<hr style=' border: 2px solid black'>
<h4 style="text-align:center;"> Stage 2</h4>
<hr style=' border: 2px solid black'>
<hr style=' border: 2px solid red'>

<p> Find a way to iterate through every infinitive and get the conjugations of every word </p>

<ol>
    <li> Scout out page structure using HTML and url</li>
    <li> Work out how to take sections of tables to rearrange in to db format</li>
    <li> Determine structure for a scraping function to crawl the entire site</li>
    
</ol>

In [3]:
# scouting tables on site
doc = url.request.urlopen('https://fi.m.wiktionary.org/wiki/Liite:Verbitaivutus/suomi/lukea').read()
soup = bs(doc,'html.parser')
soup.prettify()
tables = soup.find_all('table')

# get tables to display so they can be destructured
for table in tables:
    display(HTML(str(table)))
    print('table end \n\n\n\n')
    
    

vokaalivartalo,-
heikko vokaalivartalo,lue-
vahva vokaalivartalo,luke-
konsonanttivartalo,-


table end 






table end 






table end 






table end 






table end 






table end 






In [4]:
#scouting for particular lines/format
doc = url.request.urlopen('https://fi.m.wiktionary.org/wiki/Liite:Verbitaivutus/suomi/lukea').read()
soup = bs(doc,'html.parser')
soup.prettify()
tables = soup.find_all('table')
#table 1 
rows = tables[1].find_all('tr')
#HTML(str(rows))
rows = rows[3:4].copy()
rows[0]
#display(HTML(rows[0]))

y = []
for row in rows:
    x = []
    display(HTML(str(row)))

minä,luen,en lue,,minä,olen lukenut,en ole lukenut


<hr style=' border: 2px solid red'>
<hr style=' border: 2px solid black'>
<h4 style="text-align:center;"> Stage 3</h4>
<hr style=' border: 2px solid black'>
<hr style=' border: 2px solid red'>

<p> Now build the function to crawl the site and get verbs and their conjugates into a 3rd normal form db structure </p>

In [5]:
# function to visit Wiki 
# Extract data
#clean and extract
# return cleaned df
def scrape_verb(infinitive, english):
    # set up a data Frame to use to format data 
    # database will store ample redundant data 28 repetitions of infinitive forms to facilitate data base joins/filtering
    cleaned = pd.DataFrame({'Infinitive(eng)':[f'{english}']*28,'Infinitive(FI)':[f'{infinitive}']*28})
    
    # add the example column reminders of how tenses work
    
    cleaned['examples'] = ['I am walking',
                       'you are walking',
                       'he/she is walking',
                       'we are walking',
                       'you(plural/respectful) are walking',
                       'they are walking',
                       '(passive) walking',
                       'I walked',
                       'You walked',
                       'he/she walked',
                       'we walked',
                       'you(plural/respectful) walked',
                       'they walked',
                       '(passive) walked',
                       'I was walking',
                       'You were walking',
                       'he/she was walking',
                       'we were walking',
                       'you(plural/respectful) were walking',
                       'they were walking',
                       '(passive) were walking',
                       'I had walked',
                       'You had walked',
                       'He/she had walked',
                       'We had walked',
                       'You(plural/repectful) had walked',
                       'They had walked',
                       '(passive) had walked'
                       ]
    #open the relevant page 
    doc = url.request.urlopen(f'https://fi.m.wiktionary.org/wiki/Liite:Verbitaivutus/suomi/{infinitive}').read()
    
    #read to soup using bs4
    soup = bs(doc,'html.parser')
    soup.prettify()
    
    # find the tables on the page
    tables = soup.find_all('table')
    
    
    HTML(str(tables[1]))

    rows = tables[1].find_all('tr')

    y = []
    for row in rows:
        x = []
        data = row.find_all('td')
        #display(HTML(str(data)))
        for datum in data:
            i = str(datum).find('<br/>')
            if i>0:
                #print(i)
                x.append(bs(str(datum)[:i].replace('\n','') +'</td>').text)
            else:
                x.append(datum.text.replace('\n',''))
        if x:
            y.append(x)

    frame = pd.DataFrame(y)
    # present/future
    a = frame.iloc[0:7,0:3]
    # past
    b = frame.iloc[0:7,4:]
    # imperfect
    c = frame.iloc[7:,0:3]
    # past imperfect
    d = frame.iloc[7:,4:]

    for x in [a,b,c,d]:
        x.columns = ['person','positive','negative']
    #display(frame)

    verb = pd.concat([a,b,c,d],ignore_index = True)
    cleaned = cleaned.merge(verb, how = 'left', left_index = True, right_index = True)
    return cleaned


#test here
#scrape_verb('lukea','read')

<hr style=' border: 2px solid red'>
<hr style=' border: 2px solid black'>
<h4 style="text-align:center;"> Stage 4</h4>
<hr style=' border: 2px solid black'>
<hr style=' border: 2px solid red'>

<p> Use the function to scrape the site in blocks, this was to save waiting time as I hate waiting for code to run. The set was split in to blocks of 500 so run times were <b> &lt; 5 mins </b> per block </p>

In [5]:
#read the infinitives csv and select a range
test_group = pd.read_csv('./cleaneddb/verbinfinitives.csv')[7500:]
print(test_group.shape)
# check the table 
test_group.head()

(592, 2)


,Finnish infinitive,English inifinitive
7500,valaa,cast
7501,valahtaa,slump
7502,valaista,light
7503,valaistua,become alight
7504,valehdella,lie


<p>Run the function as well as diagnostics on the block of verbs to be scraped and formatted </p>

In [37]:
# create data frame to receive all verbs
total_df = pd.DataFrame()
i = 0
# list to catch broken links
# this was needed because some old parlance verbs are in the list 
# (I checked this with my Finnish partner to make sure nothing useful was lost)
broken_links = []

# loop through the block of verbs to be scraped and process them
for fin,eng in zip(test_group["Finnish infinitive"],test_group["English inifinitive"]):
    i+=1
    # attempt to run the scraping/formatting process
    try:
        #concat result to the overall data frame
        total_df = pd.concat([total_df,scrape_verb(fin,eng)])
    # catch any errors and add them to the list of broken links
    except:
        broken_links.append(fin)
        broken = test_group[test_group['Finnish infinitive']==fin]
        broken = broken[['English inifinitive','Finnish infinitive']]
        broken.columns = ['Infinitive(eng)','Infinitive(FI)']
        # add a simplified row to the overall data set, just a row with the infinitive and its transalation
        total_df = pd.concat([total_df,broken], ignore_index = True)
    if i%50 == 0:
        # sleep every 50 links to avoid triggering cloudflare/other security
        time.sleep(0.5)
    if i%100 == 0:
        #print every 100 to allow me to track progress
        print(i)
    

100
200
300
400
500


In [ ]:
broken_links

In [29]:
len(broken_links)

504

<hr style=' border: 2px solid red'>
<hr style=' border: 2px solid black'>
<h4 style="text-align:center;"> Stage 5</h4>
<hr style=' border: 2px solid black'>
<hr style=' border: 2px solid red'>

<p> Use the pygwalker to check data quality and fromat, pyg is an open source version of Tableau so very handy for quickly visualising data sources and data.</p>

In [ ]:
# use pygwalker to check data quality and later for some analytics
pyg.walk(total_df)

<hr style=' border: 2px solid red'>
<hr style=' border: 2px solid black'>
<h4 style="text-align:center;vertical-align: middle;"> Stage 6</h4>
<hr style=' border: 2px solid black'>
<hr style=' border: 2px solid red'>

<p> Export the block of verbs to a folder for later combination using powerQuery</p>

In [38]:
total_df.to_csv('./cleaneddb/verbs/7500-fv.csv',encoding = 'utf-8-sig')

<hr style=' border: 2px solid red'>
<hr style=' border: 2px solid black'>
<h4 style="text-align:center;"> Now to move on </h4>

<hr style=' border: 2px solid black'>
<hr style=' border: 2px solid red'>

<p> Next step is to scrape nouns and adjectives </p>

In [10]:
# change styling in notebook
HTML("""<style>
.rendered_html p{font-size:1.2em;}
.rendered_html h4{font-size:1.5em;
                 vertical-align: middle;}
</style>""")